# Saving all parquet files into a single file

This notebook is for testing, there are separate scripts for each type of data.

In [1]:
import os 
import pathlib
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import matplotlib.pyplot as plt

base_dir = pathlib.Path("../../kaggle_data/hms")
# base_dir = pathlib.Path("../../data/hms")
# base_dir = pathlib.Path("/kaggle/input/hms-harmful-brain-activity-classification")

In [2]:
path_train = base_dir / "train.csv"
df_traincsv = pd.read_csv(path_train)
df_traincsv.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


# Using pandas

### Original sub spectrograms

Pandas is involved here. Saving each spectrogram as a numpy 2D array inside a numpy array: (N, H, W)

In [35]:
# This is just code for testing, using a sample. 
# Use 00_single_original.py for the complete dataset.

train_files_specs = os.listdir(base_dir / "train_spectrograms")

df = pd.DataFrame()
for i, specfile in enumerate(train_files_specs[0:5]):
    if i%100 == 0:
        print(f'Loaded {i} files.')
    spec_id = int(specfile.split('.')[0])
    df_singlespec = pd.read_parquet(f'{base_dir}/train_spectrograms/{spec_id}.parquet')
    data = np.ones(df_singlespec.shape[0], dtype=np.int16)*spec_id
    df_singlespec.insert(0, 'spectrogram_id', data)
    df = pd.concat([df,df_singlespec], ignore_index=True)
print('Saving to specs.parquet')
df.to_parquet('../data/specs.parquet')

Loaded 0 files.
Saving to specs.parquet


Loading all spectrograms from a single parquet in only 6.5 s:

In [4]:
df_specs = pd.read_parquet('../data/00_single_spectrograms_originals.parquet')

In [5]:
df_specs.head()

,spectrogram_id,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,399182714,1,212.539993,375.029999,408.130005,347.369995,273.859985,135.289993,96.489998,82.160004,...,0.65,0.65,0.51,0.58,0.48,0.52,0.52,0.58,0.68,0.71
1,399182714,3,121.190002,205.570007,174.610001,192.529999,144.130005,76.650002,70.220001,65.050003,...,0.66,0.49,0.56,0.46,0.41,0.44,0.44,0.47,0.45,0.48
2,399182714,5,265.070007,373.109985,396.140015,369.790009,323.269989,226.000000,178.229996,174.490005,...,0.88,1.05,0.94,0.90,0.66,0.81,0.53,0.54,0.62,0.41
3,399182714,7,268.450012,287.470001,364.239990,338.390015,252.169998,278.339996,208.979996,159.399994,...,0.71,0.77,0.84,0.78,0.70,0.67,0.58,0.62,0.62,0.48
4,399182714,9,137.570007,128.470001,119.339996,135.380005,62.990002,83.089996,79.650002,61.110001,...,0.45,0.43,0.35,0.33,0.38,0.39,0.41,0.59,0.57,0.58


In [6]:
df_specs.describe()

,spectrogram_id,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
count,4.279506e+06,4.279506e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,...,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06,4.176784e+06
mean,1.070316e+09,5.442852e+02,2.519857e+04,3.100088e+04,3.448983e+04,3.028573e+04,2.612702e+04,2.255668e+04,1.965838e+04,1.689880e+04,...,7.791738e+02,7.790952e+02,7.854884e+02,8.044727e+02,8.454445e+02,9.369199e+02,1.163362e+03,2.351755e+03,4.684458e+03,5.000536e+03
std,6.229326e+08,8.775862e+02,5.414510e+05,6.673390e+05,7.481408e+05,6.835497e+05,6.008598e+05,5.297333e+05,4.707712e+05,4.143213e+05,...,1.824941e+04,1.807816e+04,1.798693e+04,1.818290e+04,1.889164e+04,2.105798e+04,2.797170e+04,6.916235e+04,1.669476e+05,1.752394e+05
min,3.537330e+05,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.283780e+08,1.930000e+02,8.950000e+00,1.173000e+01,1.299000e+01,1.153000e+01,9.310000e+00,7.220000e+00,5.620000e+00,4.450000e+00,...,5.000000e-02,5.000000e-02,5.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02,5.000000e-02,5.000000e-02,5.000000e-02
50%,1.060576e+09,3.850000e+02,2.408000e+01,3.122000e+01,3.447000e+01,3.086000e+01,2.589000e+01,2.105000e+01,1.668000e+01,1.319000e+01,...,1.400000e-01,1.400000e-01,1.400000e-01,1.300000e-01,1.300000e-01,1.300000e-01,1.300000e-01,1.400000e-01,1.500000e-01,1.500000e-01
75%,1.619851e+09,5.770000e+02,6.917000e+01,8.762000e+01,9.598000e+01,8.633000e+01,7.177000e+01,5.940000e+01,4.872000e+01,4.000000e+01,...,4.500000e-01,4.500000e-01,4.400000e-01,4.400000e-01,4.300000e-01,4.400000e-01,4.400000e-01,5.000000e-01,5.500000e-01,5.500000e-01
max,2.147388e+09,1.823100e+04,7.606394e+07,1.000231e+08,1.086720e+08,1.148422e+08,9.362598e+07,7.363883e+07,7.721102e+07,6.556956e+07,...,3.397892e+06,3.571082e+06,2.753280e+06,3.152976e+06,3.387124e+06,3.113109e+06,2.985965e+06,1.150180e+07,2.330165e+07,2.402086e+07


In [7]:
nan_rows = df_specs.isna().any(axis=1)

In [9]:
df_specs[nan_rows]

,spectrogram_id,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
1827,415275410,503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1828,415275410,505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1829,415275410,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1830,415275410,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1831,415275410,511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270587,2125601164,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4270588,2125601164,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4270589,2125601164,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4270590,2125601164,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


102k rows are NaN, I'm replacing with zero, it's fast, so it's better to replace each time than to save the modified dataframe to file.

In [10]:
df_specs.replace(np.nan, 0, inplace=True)

### Saving a reduced dataset


In [6]:
base_dir = pathlib.Path("../../kaggle_data/hms")
# base_dir = pathlib.Path("../../data/hms")
# base_dir = pathlib.Path("/kaggle/input/hms-harmful-brain-activity-classification")

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')
spectrograms = df_traincsv.spectrogram_id.unique()
len(spectrograms)


11138

In [7]:
# How many spectrograms to save?
n_specs = 800

In [10]:
spec_id = 1298803383
df_traincsv.loc[df_traincsv.spectrogram_id == spec_id]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
64474,602584638,0,0.0,1298803383,0,0.0,1899564871,49234,Seizure,3,0,0,0,0,0
64475,602584638,1,10.0,1298803383,1,10.0,1926310052,49234,Seizure,3,0,0,0,0,0
64476,602584638,2,16.0,1298803383,2,16.0,4172870566,49234,Seizure,3,0,0,0,0,0
64477,602584638,3,38.0,1298803383,3,38.0,3911037849,49234,Seizure,3,0,0,0,0,0
64478,602584638,4,40.0,1298803383,4,40.0,1846393544,49234,Seizure,3,0,0,0,0,0
64479,602584638,5,46.0,1298803383,5,46.0,21380605,49234,Seizure,3,0,0,0,0,0
64480,602584638,6,58.0,1298803383,6,58.0,753558906,49234,Other,4,0,0,0,0,15
64481,602584638,7,64.0,1298803383,7,64.0,302321986,49234,Other,4,0,0,0,0,15
64482,602584638,8,68.0,1298803383,8,68.0,166636342,49234,Seizure,3,0,0,0,0,0
64483,602584638,9,74.0,1298803383,9,74.0,1958159648,49234,Seizure,3,0,0,0,0,0


In [14]:
# Saving a new train.csv

specs_id = np.random.permutation(spectrograms)[0:n_specs]
df = pd.DataFrame()
for spec_id in specs_id:
    df = pd.concat([df, df_traincsv.loc[df_traincsv.spectrogram_id == spec_id]], ignore_index=True)

# df.to_csv('../data/reduced_ds/train.csv', index=False)    


In [15]:
# Saving all spectrograms in this reduced dataset in a single parquet files

df_specs = pd.DataFrame()
for spec_id in specs_id:
    df_singlespec = pd.read_parquet(f'{base_dir}/train_spectrograms/{spec_id}.parquet')
    data = np.ones(df_singlespec.shape[0], dtype=np.int16)*spec_id
    df_singlespec.insert(0, 'spectrogram_id', data)
    df_specs = pd.concat([df_specs,df_singlespec], ignore_index=True)
print('Saving to spectrograms_reduced_800.parquet')
# df_specs.to_parquet('../data/reduced_ds/spectrograms_reduced_800.parquet')


Saving to spectrograms_reduced_800.parquet


In [16]:
df_specs

,spectrogram_id,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,2105764400,1,2.040000,2.080000,1.860000,1.860000,2.300000,2.720000,2.460000,2.040000,...,0.69,0.66,0.39,0.48,0.39,0.44,0.45,0.370000,0.320000,0.230000
1,2105764400,3,2.260000,3.700000,5.360000,4.670000,4.470000,6.050000,4.030000,4.270000,...,0.45,0.41,0.29,0.33,0.32,0.37,0.34,0.250000,0.220000,0.280000
2,2105764400,5,4.040000,4.240000,5.750000,5.530000,3.120000,3.450000,2.730000,3.220000,...,0.53,0.41,0.38,0.38,0.35,0.16,0.17,0.160000,0.130000,0.200000
3,2105764400,7,5.100000,6.810000,6.840000,6.480000,4.220000,1.010000,1.600000,1.630000,...,0.54,0.35,0.36,0.32,0.26,0.23,0.11,0.160000,0.210000,0.190000
4,2105764400,9,3.840000,4.970000,5.100000,5.760000,3.510000,4.150000,3.700000,3.660000,...,0.37,0.26,0.15,0.20,0.15,0.15,0.22,0.270000,0.280000,0.290000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311395,654538697,607,126.900002,133.410004,176.279999,152.130005,109.290001,87.989998,51.660000,33.590000,...,0.67,1.39,1.98,1.87,1.71,2.05,13.99,515.570007,1381.959961,1383.660034
311396,654538697,609,125.019997,264.529999,290.079987,212.800003,226.619995,97.029999,57.840000,34.520000,...,0.38,1.42,2.33,2.60,1.69,2.32,1.80,673.210022,1308.310059,1374.069946
311397,654538697,611,120.690002,185.080002,140.580002,181.929993,182.389999,56.779999,32.820000,14.030000,...,0.32,0.86,3.50,4.48,1.67,1.18,2.46,642.200012,1212.930054,1308.550049
311398,654538697,613,576.109985,780.070007,732.320007,613.849976,413.959991,153.460007,33.310001,35.970001,...,0.41,0.89,2.63,2.16,2.27,1.59,2.03,710.960022,1272.420044,1334.660034


## Only numpy, no pandas for accessing the data from tensorflow

This code is for testing, using a sample. 

Use 00_single_original_numpy.py for the complete dataset.

In [3]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')
df_traincsv.loc[df_traincsv.expert_consensus == 'Seizure', 'target'] = 0
df_traincsv.loc[df_traincsv.expert_consensus == 'LPD', 'target'] = 1
df_traincsv.loc[df_traincsv.expert_consensus == 'GPD', 'target'] = 2
df_traincsv.loc[df_traincsv.expert_consensus == 'LRDA', 'target'] = 3
df_traincsv.loc[df_traincsv.expert_consensus == 'GRDA', 'target'] = 4
df_traincsv.loc[df_traincsv.expert_consensus == 'Other', 'target'] = 5

print("Loaded train.csv. Added target column.")

Loaded train.csv. Added target column.


### Saving reduced numpy dataset

The same length (800 spectrogram files).

In [4]:
# For definitive code, change:
# - Slice of data, to include all files.
# - Increase divisor in remainder check.
# - Name of output files.

train_files_specs = os.listdir(f'{base_dir}/train_spectrograms')

# Saving columns of interest
specfile = train_files_specs[0]
spec_id = int(specfile.split('.')[0])
df_singlespec = pd.read_parquet(f'{base_dir}/train_spectrograms/{spec_id}.parquet')
columns = df_singlespec.columns[1:]

all = np.array([]).reshape(0,400)
sub_spectrograms_idxs = np.array([], dtype=int).reshape(0,4)
current = 0

for n, specfile in enumerate(train_files_specs[0:800]):
    if n%40 == 0:
        print(f'{n} files loaded.')
    spec_id = int(specfile.split('.')[0])
    df_singlespec = pd.read_parquet(f'{base_dir}/train_spectrograms/{spec_id}.parquet')
    df_singlespec.replace(np.nan, 0, inplace=True)
    length = df_singlespec.shape[0]

    subs = df_traincsv.loc[df_traincsv.spectrogram_id == spec_id]
    for i in np.arange(len(subs)):
        item = subs.iloc[i]
        eeg_id = item.eeg_id
        eeg_sub_id = item.eeg_sub_id
        idx = current + int(item.spectrogram_label_offset_seconds/2)
        target = int(item.target)
        x = np.array([eeg_id, eeg_sub_id, idx, target]).reshape(1,4)
        sub_spectrograms_idxs = np.concatenate([sub_spectrograms_idxs, x])        
    x = df_singlespec[columns].to_numpy(copy=True)
    all = np.concatenate([all, x])
    current = current + x.shape[0]


np.save("../data/00_sub_spectrograms_idxs_reduced.npy", sub_spectrograms_idxs)
np.save("../data/00_spectrograms_reduced.npy", all)

0 files loaded.


KeyboardInterrupt: 

Testing that the spectrograms in the npy files are equal to the spectrograms in the parquet files.

In [75]:
items = np.load("../data/00_sub_spectrograms_idxs_reduced.npy")
specs = np.load("../data/00_spectrograms_reduced.npy")

In [97]:
# Columns of interest.
df = pd.read_parquet(f'{base_dir}/train_spectrograms/399182714.parquet')
columns = df.columns[1:]

spectrogram_id = 0
for i in np.arange(items.shape[0]):
    if i%200 == 0:
        print(f'{i} items tested.')
    item_train = df_traincsv.loc[(df_traincsv.eeg_id == items[i][0])&(df_traincsv.eeg_sub_id == items[i][1])].iloc[0]
    if item_train.spectrogram_id != spectrogram_id:
        spectrogram_id = item_train.spectrogram_id
        df = pd.read_parquet(f'{base_dir}/train_spectrograms/{spectrogram_id}.parquet')
        df.replace(np.nan, 0, inplace=True)

    offset = item_train.spectrogram_label_offset_seconds
    original_spec = df.loc[(df.time >= offset)&(df.time < (offset + 600))][columns].to_numpy(copy=True)
    idx = items[i][2]
    saved_spec = specs[idx:(idx+300)]
    if ~np.all(saved_spec == original_spec):
        print("Fail:", i, items[i], spectrogram_id)


0 items tested.
200 items tested.
400 items tested.
600 items tested.
800 items tested.
1000 items tested.
1200 items tested.
1400 items tested.
1600 items tested.
1800 items tested.
2000 items tested.
2200 items tested.
2400 items tested.
2600 items tested.
2800 items tested.
3000 items tested.
3200 items tested.
3400 items tested.
3600 items tested.
3800 items tested.
4000 items tested.
4200 items tested.
4400 items tested.
4600 items tested.
4800 items tested.
5000 items tested.
5200 items tested.
5400 items tested.
5600 items tested.
5800 items tested.
6000 items tested.
6200 items tested.
6400 items tested.
6600 items tested.
6800 items tested.
7000 items tested.
7200 items tested.
7400 items tested.


## Fixing items:

Removing sub eegs with NaN in sub spectrograms.

In [6]:
idxs_sub_specs_with_nans = np.load("../data/idxs_sub_specs_with_nans.npy")
items = np.load("../data/00_single_spectrograms_originals_numpy_items.npy")
df_items = pd.DataFrame(items, columns=['eeg_id', 'eeg_sub_id', 'row', 'target'])
# df_traincsv.index.difference(idxs_sub_specs_with_nans)
df_items

,eeg_id,eeg_sub_id,row,target
0,2619655982,0,0,5
1,2619655982,1,1,5
2,2619655982,2,3,5
3,2619655982,3,24,5
4,2547171910,0,324,5
...,...,...,...,...
106795,597018294,15,4279145,2
106796,597018294,16,4279167,0
106797,597018294,17,4279175,0
106798,3434520650,0,4279202,0


In [13]:
idx_to_remove = np.array([], dtype=int)
for i in np.arange(len(idxs_sub_specs_with_nans)):
    sub = df_traincsv.iloc[idxs_sub_specs_with_nans[i]]
    idx = df_items.loc[(df_items.eeg_id == sub.eeg_id)&(df_items.eeg_sub_id == sub.eeg_sub_id)].index
    idx_to_remove = np.append(idx_to_remove, idx)

idx_to_remove

array([    30,     31,     32, ..., 106413, 106414, 106415])

In [14]:
len(idx_to_remove)

7743

In [15]:
idxs = df_items.index.difference(idx_to_remove)
len(idxs)

99057

In [16]:
df_items = df_items.iloc[idxs]
np.save("../data/00_single_spectrograms_originals_numpy_items_v2.npy", df_items.to_numpy())

## Saving as parquet

In [9]:
matrix = np.load("../data/00_spectrograms_reduced.npy")
# matrix = np.load("../data/00_sub_spectrograms_idxs_reduced.npy")

arrays = [
    pa.array(col)  # Create one arrow array per column
    for col in matrix
]

table = pa.Table.from_arrays(
    arrays,
    names=[str(i) for i in range(len(arrays))] # give names to each columns
)
# Save it:
pq.write_table(table, 'table.parquet')



In [6]:
# Read it back as numpy:
table_from_parquet = pq.read_table('table.parquet')
matrix_from_parquet = table_from_parquet.to_pandas().T.to_numpy()

In [7]:
np.all(matrix_from_parquet == matrix)

True

In [8]:
matrix_from_parquet

array([[3403533082,          0,          0,          3],
       [3403533082,          1,          1,          3],
       [3403533082,          2,          2,          3],
       ...,
       [  72664070,          5,       6516,          0],
       [3155112843,          0,       6496,          5],
       [2448761963,          0,       6496,          1]])